In [ ]:
import sys
sys.path.append('/mnt/MarketData/JSON')

# Requirements

## High level
- Must model ELGD (Expected Loss Given Default)
- Must model risk neutral PD's (assuming the ELGD above)
- Must model discounted expected exposure

## Modelling
- Must model collateral with MTA's/Thresholds correctly (**done**)
- Must assume a Margin Period of Risk of 4+N for cleared and SFT's 9+N days for everything else where N is the margining frequency (N=1 for daily) (**done**)
- Must use risk neutral (i.e. market implied) drifts and vols for all simulation paths where available (only falling back to historical values if there are none) (**mostly done - needs ICIB IT to maintain new vol surfaces**)
- Simulation should allow for non-normal exposure distribution (i.e. fat tails) (**implicitly done**)
- Take into account WWR (Wrong Way Risk) i.e. correlation of exposure to PD (**not done**)
- Multiplier
    - The results of the SACVA calc must be scales with the multiplier (lowest value is 1.0 - can be increased at the regulators discretion)
- Delta and Vega must be calculated across all asset classes except for Counterparty Credit Risk (where only Delta is required)
    - This means we need to calculate vega risk for all reference credit names (**not done**)

## Input Data 
- Credit spread curves need to be maintained and updated
- Buckets for Equities, Commodities and Counterparty mapping needs to be maintained 
- Counterparties that have a legal relationship to one another need to clearly mentioned (they attract a different correlation coefficient later - **TODO**)

# What we need to do to meet the above

## Modelling TODO
- Need to model credit spreads for all Reference Names (to calculate vega on Credit reference names)
    - Need a measure of volatility for all reference names 
        - can use historical vols of spreads if none available (probably will be the case)
        - this would affect names like natwest
- WWR
    - Can be modelled but is quite hard - need historical PD's to calibrate to (and we've not had a TSDB history of credit spreads since forever)

    
## Stuff we need to check with the Regulator
- Vols for Equities/IR/Commodities/FX are expressed as being relative
    - i.e. the sensitivity of the calc is expressed as 1% relative to the current vol and then multiplied by 100 (i.e. / 0.01)
    - e.g. if the fx vol is 10%, then the sensitivity is calculated as $100*(CVA_{10.1\%}-CVA_{10\%})$ as opposed to $100*(CVA_{11\%}-CVA_{10\%})$
        - Note that $100*(CVA_{11\%}-CVA_{10\%})$ is closer to the definition of the derivative that a relative shift $\Big(\frac{f(x+0.01)-f(x)}{0.01}\Big)\approx f'(x)$ 
        - And $100*(CVA_{10.1\%}-CVA_{10\%})$ is dependent on the current value of the vol i.e. would be calculated as $100*f'(x)*0.01*x = x*f'(x)$ which, (if vols<100%) is considerably smaller 
            - happy to do this but maybe let the regulator know that this is not consistent with other measures

In [ ]:
# standard libs
import os
import glob
import random
import operator
import numpy as np
import pandas as pd

from functools import reduce
# need to strip out non-digits or non-ascii
from string import digits, ascii_letters
# need to read vega file
import riskflow as rf
# import riskflow.utils as utils
# from riskflow.adaptiv import AdaptivContext
from riskflow.riskfactors import construct_factor

# formatting and setup
%matplotlib notebook
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200

# get all the sensitivities
market_data_dir = '/mnt/MarketData/CVAMarketDataBackup'
base_dir = '/mnt/xva/SACVA'
sensitivities_dir = os.path.join(base_dir, 'Greeks')
output_dir = '.'


# Change the folder/rundate here
with open('/mnt/xva/RunDate.txt', 'rt') as f:
    rundate = f.read()
    
rundate='2024-01-26'
print('running SACVA aggregation for {}'.format(rundate))

In [ ]:
# get the short date
short_date = ''.join(rundate[2:].split('-')[::-1])
# load the marketdata and the vega marketdata
base_md = rf.load_market_data('', market_data_dir, json_name='CVAMarketData_Calibrated_New_{}.json'.format(''.join(short_date)))
vega_md = rf.load_market_data('', market_data_dir, json_name='CVAMarketData_Calibrated_Vega_{}.json'.format(''.join(short_date)))


if 'HullWhite2FactorModelParameters.USD-OIS' not in base_md.params['Price Factors']:
    base_md.params['Price Factors']['HullWhite2FactorModelParameters.USD-OIS'] = base_md.params['Price Factors']['HullWhite2FactorModelParameters.USD-SOFR']
    
if 'HullWhite2FactorModelParameters.USD-OIS' not in vega_md.params['Price Factors']:
    vega_md.params['Price Factors']['HullWhite2FactorModelParameters.USD-OIS'] = vega_md.params['Price Factors']['HullWhite2FactorModelParameters.USD-SOFR']

## EQ, FX, CM and IR Vega

For IR Vega per currency (and Commodity Vega), the change in the calibration parameters is also loaded up and the CVA change is calculated (and then divided by 1%). Note that currently the gradient is calculated based on an absolute shift of 1% implied vol (the regs say 1% relative shift but that sounds wrong)

For FX and EQ vega, we already have an ATM GBM vol term structure so we just need to shift it directly by 1% (done later in the apply_vega function)

In [ ]:
def calc_implied_vega_diff(vega_params):
    deltas = []
    for param in vega_params:
        delta = []
        index = []
        factor = param.split('.')
        zero = construct_factor(
            rf.utils.Factor(factor[0], (factor[1],)), base_md.params['Price Factors'], base_md.params['Price Factor Interpolation'])
        vega = construct_factor(
            rf.utils.Factor(factor[0], (factor[1],)), vega_md.params['Price Factors'], vega_md.params['Price Factor Interpolation'])

        z = zero.current_value()
        v = vega.current_value()
        for k,tenor in vega.get_tenor_indices().items():
            index.extend(list(zip([k]*len(tenor),tenor.reshape(-1))))
            if k in v:
                delta.extend(v[k]-z[k])
            else:
                delta.extend([None])
        deltas.append(pd.DataFrame(delta, index=pd.MultiIndex.from_tuples(index), columns=[factor[1]]))
    
    # now we know how the calibration parameters changed for a 1% bump in implied vols (i.e. impact of calibration)
    Vega = pd.concat(deltas, axis=1).T
    return Vega.T.to_dict()

# grab the vega parameters for IR and CM
ir_vega_params = [x for x in vega_md.params['Price Factors'].keys() if x.startswith('HullWhite2FactorModelParameters')]
cm_vega_params = [x for x in vega_md.params['Price Factors'].keys() if x.startswith('CSForwardPriceModelParameters')]
IR_Vega_Lookup = calc_implied_vega_diff(ir_vega_params)
CM_Vega_Lookup = calc_implied_vega_diff(cm_vega_params)



In [ ]:
# check the percentage difference in vega parameters for interest rates (should be small)
100*pd.DataFrame(IR_Vega_Lookup)

## Load up parameters

- Load up Risk weights by risk type (FX, Commodity, Counterparty, Equity, Interest Rate)
- Load up Correlations for some risk types

In [ ]:
domestic_ccy = 'ZAR'
CR_cpy_correlation = pd.read_csv('static/FRTB_CR_Cpy_Bucket_Correlations.csv', index_col=0)
CR_cpy_buckets = pd.read_csv('static/FRTB_CR_Cpy.csv', index_col=1)
CR_ref_correlation = pd.read_csv('static/FRTB_CR_Ref_Correlation.csv', index_col=0)
CR_ref_buckets = pd.read_csv('static/FRTB_CR_Ref.csv', index_col=0)

CM_buckets = pd.read_csv('static/FRTB_CM_Buckets.csv', index_col=0)
CM_map = pd.read_csv('static/comm.csv', index_col=0)
CM_lookup = CM_map['Class'].to_dict()

EQ_buckets = pd.read_csv('static/FRTB_EQ_Buckets.csv', index_col=0)
EQ_bucket_map = EQ_buckets['Equity Group'].to_dict()
EQ_Reverse_Buckets = EQ_buckets.reset_index().set_index('Equity Group')['Bucket'].to_dict()
EQ_map = pd.read_csv('static/SIMM_Buckets.csv')
EQ_lookup = EQ_map.set_index(EQ_map['FA Stock Name'].apply(lambda x: x.strip().replace('/', '_')))['SIMM Bucket'].apply(lambda x:EQ_bucket_map[int(x) if not x.startswith(' ') else 11]).to_dict()
CR_Credit_Spread = pd.read_csv('static/CP_mapping.csv', index_col=0)

IR_Delta_Correlation = pd.read_csv('static/FRTB_IR_Correlation.csv', index_col=0)
IR_Delta_RW = pd.read_csv('static/FRTB_IR_RW.csv', index_col=0)
IR_Delta_Curr = [domestic_ccy]+pd.read_csv('static/FRTB_IR_RW_CCY.csv').columns.to_list()
IR_Delta_Tenors = pd.to_numeric(IR_Delta_RW.index[:-1], errors='ignore').values
CR_Delta_Tenors = np.array([.5, 1, 3, 5, 10])

In [ ]:
no_digits = str.maketrans('', '', digits)
no_alpha  = str.maketrans('', '', ascii_letters)

CR_Credit_Spread = pd.read_csv('static/CP_mapping.csv', index_col=0)

CR_bucket_map = {'1a':'Sovereigns_a',
                 '1b':'Sovereigns_b',
                 '2':'Financials',
                 '3':'Industrials',
                 '4':'Consumer Goods',
                 '5':'Technology',
                 '6':'Utilities',
                 '7':'Other', 
                 '8':'QI'}

CR_Credit_Spread['Code']=CR_Credit_Spread.apply(lambda x:('IG_' if x['Bucket']=='IG' else 'HYNR_')+CR_bucket_map[x['Reg CVA Bucket']], axis=1)
Kudsai = CR_Credit_Spread['Code'].to_dict()

In [ ]:
# need to interpolate the IR curves at the tenors specified

def calc_bucket_sensi(delta_tenors, val):
    # old ir bucket calc - uses little sliding triangular shifts to estimate the delta
    # has since been superceeded by ir_bucket_sensi (see below)
    max_tenor = val.index.max()
    deltas = delta_tenors[:delta_tenors.searchsorted(max_tenor)+1]
    x = val.index.values
    y = val.iloc[:,-1].values
    bump = []
    prev_tenor = 0.0    
    for index, tenor in enumerate(deltas):        
        prev_piece = (x>=prev_tenor)&(x<=tenor)
        next_tenor = delta_tenors[index+1] if index<delta_tenors.size-1 else max_tenor
        next_piece = (x>=tenor)&(x<=next_tenor)
        prev_sensi = (np.interp(x[prev_piece], [prev_tenor,tenor],[0.0, 0.0001])*y[prev_piece]).sum()
        next_sensi = (np.interp(x[next_piece], [tenor,next_tenor],[0.0001, 0.0])*y[next_piece]).sum()
        prev_tenor = tenor
        bump.append((prev_sensi+next_sensi)/0.0001)
    return bump, deltas

def ir_bucket_sensi(delta_tenors, val):
    # this is just a straight sum per bucket
    max_tenor = val.index.max()
    deltas = delta_tenors[:delta_tenors.searchsorted(max_tenor) + 1]
    bump = val.groupby(pd.cut(val.index, bins=np.append([0], deltas)), observed=True).sum().values[:, -1]
    return bump, deltas

def calc_cs01_sensi(cr_tenors, val):
    max_tenor = val.index.max()
    cs01_clipped = cr_tenors[cr_tenors.index <= max_tenor]
    max_effective_cds_tenor = CR_Delta_Tenors.searchsorted(max_tenor) + 1
    # can do this because if a tenor is not in val, then it must be zero
    bump = cs01_clipped.reindex(val.index).iloc[:,:max_effective_cds_tenor] * val.iloc[:, -1].values.reshape(-1, 1)
    return bump.sum() / 0.0001, CR_Delta_Tenors[:max_effective_cds_tenor]
    
def interpolate_IR_Curves(df, cs01, crb_name):
    q = df.set_index(['Rate','Tenor'])
    md = []
    for rate in q.index.levels[0]:
        val = q.loc[rate]
        cpy = val.columns[-1]
        name = (rate.split('/')[1] if '/' in rate else rate).split('.')
        if name[0].startswith('InterestRate') or name[0].startswith('InflationRate'):
            if name[1][:3] in IR_Delta_Curr:
                bump, ir_deltas = ir_bucket_sensi(IR_Delta_Tenors, val)
                r=pd.DataFrame({'Rate':rate, 'Tenor2':0, 'Tenor3':0, cpy:bump}, index=ir_deltas )
                r.index.name='Tenor'
            else:
                # just a straight sum
                r=pd.DataFrame({'Rate':rate, 'Tenor2':0, 'Tenor3':0, cpy:val.iloc[:,-1].sum()}, index=[0.0] )
                r.index.name='Tenor'
                
        elif name[0].startswith('SurvivalProb'):
            # need to translate the shift in negative log survival Probability
            # to a shift in credit spreads (i.e. the cs01)
            
            bump, cr_deltas = calc_cs01_sensi(cs01, val)
            new_rate = 'SurvivalProb.{}'.format(crb_name)
            r = pd.DataFrame(
                {'Rate': new_rate, 'Tenor2': 0, 'Tenor3': 0, cpy: bump.values},
                index=cr_deltas)
            r.index.name = 'Tenor'
        else:
            r = val
            r['Rate']=rate

        md.append(r)

    return pd.concat(md, sort=True).reset_index().set_index(['Rate','Tenor','Tenor2','Tenor3']) if md else df

cva_default = pd.read_csv( os.path.join(base_dir, 'Stats','SA_CVA_Stats_{}_Total.csv'.format(rundate)), index_col=0)


In [ ]:
delta={}
for crb in glob.glob(os.path.join(sensitivities_dir, 'SACVA_'+rundate+'*.csv')):
    filename = os.path.split(crb)[1]    
    crb_name = filename[filename.find('CrB_'):-4]
    # if crb_name!='CrB_ACWA_Power_SolarReserve_Redstone_So_NonISDA':
    #    continue
    cs01_name = 'CS01_{}_{}.csv'.format(rundate, crb_name)
    if os.path.isfile(os.path.join(sensitivities_dir, cs01_name)):
        cs01 = pd.read_csv(os.path.join(sensitivities_dir, cs01_name), index_col=0, skiprows=1)
    else:
        cs01 = None
    greeks = pd.read_csv(crb).iloc[:,:-1]
    try:
        delta[crb_name]= interpolate_IR_Curves(greeks, cs01, crb_name)
    except:
        print('skipping {}'.format(crb) )

## Tenor Buckets

- IR data needs to be bucketed into the following (for Domestic, USD,JPY, EUR, GBP, AUD, CAD, SEK) :
  - 1 year
  - 2 years
  - 5 years
  - 10 years
  - 30 years
  
- Counterparty spread data needs the following buckets:
  - 6 months
  - 1 year
  - 3 years
  - 5 years
  - 10 years
  
FX, Commodities and Equites do not have tenors  

In [ ]:
def apply_vega(row):
    if row.name[0].startswith('HullWhite2FactorModelParameters'):
        rate = row.name[0].split('.')
        return 100.0 * IR_Vega_Lookup[rate[1]].get((rate[-1], row.name[1]), 0.0) * row[0]

    elif row.name[0].startswith('CSForwardPriceModelParameters'):
        rate = row.name[0].split('.')
        return 100.0 * CM_Vega_Lookup[rate[1]].get((rate[-1], row.name[1]), 0.0) * row[0]

    elif row.name[0].startswith('GBMAssetPriceTSModelParameters'):
        rate = row.name[0].split('.')
        return 100.0 * (0.01 if rate[-1] == 'Vol' else 0.0) * row[0]

    else:
        return row[0]
    
def bucket_ir(y):
    if y==1:
        return '1'
    elif y==2:
        return '2'
    elif y==5:
        return '5'
    elif y==10:
        return '10'
    elif y==30:
        return '30'
    else:
        return 'all'
    
def bucket_cr(y):
    if y==.5:
        return '6 months'
    elif y==1:
        return '1 year'
    elif y==3:
        return '3 year'
    elif y==5:
        return '5 year'
    else:
        return '10 year'
    
def classify(x):
    name = x['Rate'].split('.')
    if name[0] == 'HullWhite2FactorModelParameters':
        return 'Vega.IR.' + name[1][:3]
    elif name[0] == 'GBMAssetPriceTSModelParameters':
        if len(name[1]) == 3:
            return 'Vega.FX.{}.USD'.format(name[1])
        else:
            eq = name[1].translate(no_digits)
            return 'Vega.EQ.' + EQ_lookup.get(eq, 'Other sector')
    elif name[0] == 'CSForwardPriceModelParameters':
        com = name[1]
        return 'Vega.CM.' + CM_lookup[com]
    elif name[0] == 'InterestRate':
        curr = name[1][:3]
        return 'Delta.IR.' + curr + '.' + bucket_ir(x['Tenor'])
    elif name[0] == 'InflationRate':
        curr = name[1][:3]
        return 'Delta.IR.' + curr + '.Inflation'
    elif name[0] == 'FxRate':
        curr = name[1][:3]
        return 'Delta.FX.' + curr + '.USD'
    elif name[0] == 'SurvivalProb':
        sector = Kudsai.get(name[1], 'HYNR_Other')
        return 'Delta.CR.' + sector + '.' + bucket_cr(x['Tenor'])
    elif name[0] == 'FXVol':
        curr = name[1:]
        return 'Vega.FX.' + '.'.join(curr)
    elif name[0] == 'InterestRateVol':
        curr = name[1][:3]
        return 'Vega.IR.' + curr
    elif name[0] == 'ForwardPrice':
        com = name[1]
        return 'Delta.CM.' + CM_lookup[com]
    elif name[0] == 'ForwardPriceVol':
        com = name[1]
        return 'Vega.CM.' + CM_lookup[com]
    elif name[0] == 'EquityPrice':
        eq = name[1].translate(no_digits)
        return 'Delta.EQ.' + EQ_lookup.get(eq, 'Other sector')
    elif name[0] == 'EquityPriceVol':
        eq = name[1].translate(no_digits)
        return 'Vega.EQ.' + EQ_lookup.get(eq, 'Other sector')

FRTB_buckets = {}        
for k,z in delta.items():
    if z.values.shape[0]:
        # strip out the fx implied
        # v = z[~z.index.get_level_values(0).str.startswith('GBMAssetPriceTSModelParameters')].copy()
        v = z.copy()
        v.iloc[:,0] = v.apply(apply_vega, axis=1)
        v['FRTB_Buckets'] = v.index.to_frame().apply(classify, axis=1)
        FRTB_buckets[k]=v.groupby('FRTB_Buckets').agg(np.sum)

In [ ]:
all_buckets = pd.concat(FRTB_buckets.values(), axis=1, sort=True)

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option("display.max_rows", 500, "display.max_columns", 20)
# all_buckets.to_csv('/mnt/MarketData/ab.csv')
all_buckets#['CrB_ACWA_Power_SolarReserve_Redstone_So_NonISDA']

In [ ]:
# delta['CrB_ACWA_Power_SolarReserve_Redstone_So_NonISDA']

## Vega FX

- FX has no term structure with a risk weight of $100\%$

## Vega IR

- IR sensitivity is obtained by sensitivity to the Calibration parameters (not the vol surface) - so we need to scale this down
 - has a risk weight of $100\%$
 
## Vega CM

 - CM  has a risk weight of $100\%$
 
## Vega EQ

 - EQ has a risk weight of $78\%$ for large cap buckets and $100\%$ for everything else

In [ ]:
vega_fx = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.FX')]].sum(axis=1)
vega_ir = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.IR')]].sum(axis=1)
vega_cm = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.CM')]].sum(axis=1)
vega_eq = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.EQ')]].sum(axis=1)

# constants as laid out above
R = 0.01 # Hedge disallowance
vega_IR_RW = 1.0
vega_FX_RW = 1.0
vega_CM_RW = 1.0
vega_EQ_L_RW = .78
vega_EQ_S_RW = 1.0

In [ ]:
vega_ir_crb = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Vega.IR')]].sum(axis=0)
vega_ir_crb.sort_values()

In [ ]:
bucket_vega_ir = pd.DataFrame({'SA-WS (CVA)':vega_IR_RW*vega_ir, 'SA-WS (Hedge)':vega_IR_RW*0.0}).set_index(vega_ir.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_ir['SA-WS']=bucket_vega_ir['SA-WS (CVA)']+bucket_vega_ir['SA-WS (Hedge)']
bucket_vega_ir['K_b']=np.sqrt(bucket_vega_ir['SA-WS']**2 + R*(bucket_vega_ir['SA-WS (Hedge)']**2))
bucket_vega_ir

In [ ]:
bucket_vega_fx = pd.DataFrame({'SA-WS (CVA)':vega_FX_RW*vega_fx, 'SA-WS (Hedge)':vega_FX_RW*0.0}).set_index(vega_fx.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_fx['SA-WS']=bucket_vega_fx['SA-WS (CVA)']+bucket_vega_fx['SA-WS (Hedge)']
bucket_vega_fx['K_b']=np.sqrt(bucket_vega_fx['SA-WS']**2 + R*(bucket_vega_fx['SA-WS (Hedge)']**2))
bucket_vega_fx

In [ ]:
bucket_vega_cm = pd.DataFrame({'SA-WS (CVA)':vega_CM_RW*vega_cm, 'SA-WS (Hedge)':vega_CM_RW*0.0}).set_index(vega_cm.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_cm['SA-WS']=bucket_vega_cm['SA-WS (CVA)']+bucket_vega_cm['SA-WS (Hedge)']
bucket_vega_cm['K_b']=np.sqrt(bucket_vega_cm['SA-WS']**2 + R*(bucket_vega_cm['SA-WS (Hedge)']**2))
bucket_vega_cm

In [ ]:
vega_EQ_RW = vega_eq.index.map(lambda x:vega_EQ_L_RW if x.split('.')[2].startswith('L') else vega_EQ_S_RW)
bucket_vega_eq = pd.DataFrame({'SA-WS (CVA)':vega_EQ_RW*vega_eq, 
                               'Risk Weight':vega_EQ_RW,
                               'SA-WS (Hedge)':vega_EQ_RW*0.0}).set_index(vega_eq.index.map(lambda x:x.split('.',2)[2]))
bucket_vega_eq['SA-WS']=bucket_vega_eq['SA-WS (CVA)']+bucket_vega_eq['SA-WS (Hedge)']
bucket_vega_eq['K_b']=np.sqrt(bucket_vega_eq['SA-WS']**2 + R*(bucket_vega_eq['SA-WS (Hedge)']**2))
bucket_vega_eq


##  Delta

Now calculate all the delta contributions for each risk factor type


###  Commodity Delta

- All CM Need to be bucketed as per the regs

In [ ]:
CM_RW = CM_buckets.set_index('Commodity Group').to_dict()['Risk Weight']
delta_cm = pd.DataFrame({'Delta_CM':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.CM')]].sum(axis=1)})
delta_cm['RiskWeight'] = delta_cm.apply(lambda x:CM_RW[x.name.split('.')[2]], axis=1)

# todo - add hedge
delta_cm['SA-WS (Hedge)'] = 0.0
delta_cm['SA-WS (CVA)'] = delta_cm['RiskWeight']*delta_cm['Delta_CM']
delta_cm['SA-WS'] = delta_cm['SA-WS (CVA)']+delta_cm['SA-WS (Hedge)']

In [ ]:
bucket_delta_cm = delta_cm.set_index(delta_cm.index.map(lambda x:x.split('.',2)[2]))
bucket_delta_cm['K_b']=np.sqrt(bucket_delta_cm['SA-WS']**2 + R*(bucket_delta_cm['SA-WS (Hedge)']**2))
bucket_delta_cm

###  Equity Delta

- All EQ Need to be bucketed as per the regs

In [ ]:
EQ_RW = EQ_buckets.set_index('Equity Group').to_dict()['Risk Weight']
delta_eq = pd.DataFrame({'Delta_EQ':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.EQ')]].sum(axis=1)})
delta_eq['RiskWeight'] = delta_eq.apply(lambda x:EQ_RW[x.name.split('.')[2]], axis=1)
delta_eq['SA-WS (Hedge)'] = 0.0
delta_eq['SA-WS (CVA)'] = delta_eq['RiskWeight']*delta_eq['Delta_EQ']
delta_eq['SA-WS'] = delta_eq['SA-WS (CVA)']+delta_eq['SA-WS (Hedge)']

In [ ]:
bucket_delta_eq = delta_eq.set_index(delta_eq.index.map(lambda x:x.split('.',2)[2]))
bucket_delta_eq['K_b']=np.sqrt((1-0.01)*bucket_delta_eq['SA-WS']**2 +
                               0.01*(bucket_delta_eq['SA-WS (Hedge)']**2+bucket_delta_eq['SA-WS (CVA)']**2))
bucket_delta_eq

###  FX Delta

- All fx delta's have the same risk weight (.11)
- bucketed by currency pair
- must be relative to the reporting currency (here ZAR)
- based off a 1% relative shift

In [ ]:
delta_fx = pd.DataFrame({'Delta_FX':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.FX')]].sum(axis=1)})
delta_fx['RiskWeight']=.11
delta_fx['SA-WS (Hedge)']=0.0
delta_fx['SA-WS (CVA)']=delta_fx['RiskWeight']*delta_fx['Delta_FX']
delta_fx['SA-WS']=delta_fx['SA-WS (CVA)']+delta_fx['SA-WS (Hedge)']

In [ ]:
bucket_delta_fx = delta_fx.set_index(delta_fx.index.map(lambda x:x.split('.',2)[2]))
bucket_delta_fx['K_b']=np.sqrt(bucket_delta_fx['SA-WS']**2 + R*(bucket_delta_fx['SA-WS (Hedge)']**2))
bucket_delta_fx

In [ ]:
delta_cr['CrB_ACWA_Power_SolarReserve_Redstone_So_NonISDA']


## Counterparty Delta

- need to group counterparties into the following buckets:
  - HYNR_Consumer Goods
  - HYNR_Financials	
  - HYNR_Industrials
  - HYNR_Sovereigns	(a and b)
  - HYNR_Technology	
  - HYNR_Utilities	
  - HYNR Other
  - IG_Consumer Goods
  - IG_Financials	
  - IG_Industrials	
  - IG_Sovereigns (a and b)
  - IG_Technology	
  - IG_Utilities	
  - IG Other

- Then need to apply correlation within tenors (for buckets 1-7):
    - $p_{tenor}$ is 100% if two tenors are the same, 90% otherwise
    - $p_{name}$ is 100% if two names are the same, 90% if distint but legally related (TODO), 50% otherwise
    - $p_{quality}$ is 100% if two names have the same quality (IG/IG or HYNR/HYNR), 80% otherwise
    - $p_{kl}=p_{tenor}*p_{name}*p_{quality}$ 
    
- currently ignoring legally related entities and bucket 8

- Then apply the Hedge disallowance parameter (0.01)

In [ ]:
delta_cr = all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.CR')]]

def CR_RW(row):
    bucket = row.name.split('.')[2]
    return CR_cpy_buckets.loc[bucket]['Risk Weight (RW)']
    
def CR_reindex(df):
    return df.set_index(
        pd.MultiIndex.from_arrays(
            [df.index.map(lambda x:x.split('.')[2]),df.index.map(lambda x:'.'.join(x.split('.')[3:]))]))
    
CR_SA_WS_CVA = delta_cr.apply(CR_RW, axis=1).values.reshape(-1,1)*delta_cr
CR_SA_WS_hedge = 0.0 * delta_cr
CR_SA_WS = CR_SA_WS_CVA+CR_SA_WS_hedge

delta_cr_index=CR_reindex(CR_SA_WS)
delta_cr_hedge_index=CR_reindex(CR_SA_WS_hedge)
delta_cr_CVA_index=CR_reindex(CR_SA_WS_CVA)

# get the bucket maps
cp_buckets_map = {}
for i in delta_cr_index.index.levels[0]:
    cp_buckets_map.setdefault(i.split('_',1)[1],[]).append(i)

all_delta_cr = {}

for CP_group, CP_Classifications in cp_buckets_map.items():
    group0 = delta_cr_index.xs(CP_Classifications[0]).dropna(how='all', axis=1)
    
    if len(CP_Classifications)>1:
        group1=delta_cr_index.xs(CP_Classifications[1]).dropna(how='all', axis=1)
        tenor_index = np.union1d(group0.index, group1.index)
        delta_cp_size = np.array([group0.reindex(tenor_index).size,group1.reindex(tenor_index).size])//tenor_index.size
    else:        
        tenor_index = group0.index
        delta_cp_size = np.array([group0.size])//tenor_index.size
        
    corr_cp      = []
    del_cp       = []
    del_cp_cva   = []
    del_cp_hedge = []
    correlation  = []
    
    for index, CP_Classification in enumerate(CP_Classifications):
        tenor_size = tenor_index.size
        delta_cp = delta_cr_index.xs(CP_Classification).dropna(
            how='all', axis=1).reindex(tenor_index).fillna(0.0).values.reshape(-1,1)
        delta_cp_hedge = delta_cr_hedge_index.xs(CP_Classification).dropna(
            how='all', axis=1).reindex(tenor_index).fillna(0.0).values.reshape(-1,1)
        delta_cp_cva = delta_cr_CVA_index.xs(CP_Classification).dropna(
            how='all', axis=1).reindex(tenor_index).fillna(0.0).values.reshape(-1,1)
        
        del_cp.append(delta_cp)
        del_cp_hedge.append(delta_cp_hedge)
        del_cp_cva.append(delta_cp_cva)

    # matrix for diagonal
    diag = np.ones((tenor_size,tenor_size)) * .9
    np.fill_diagonal(diag,1)

    # matrix for off diagonal - same Group
    off_diag_same = np.ones((tenor_size,tenor_size)) * .9 * .5
    np.fill_diagonal(off_diag_same, .5)

    # matrix for off diagonal - different Group
    off_diag_other = np.ones((tenor_size,tenor_size)) * .9 * .5 * .8
    np.fill_diagonal(off_diag_other, .5 * .8)

    for i in range(sum(delta_cp_size)):
        rows = [diag if j==i else (off_diag_same if i<delta_cp_size[0] else off_diag_other) for j in range(sum(delta_cp_size))]
        correlation.append(np.concatenate(rows, axis=1))
        
    corr = np.concatenate(correlation, axis=0)
    delta_cp_full = np.concatenate(del_cp,axis=0)
    delta_cp_cva_full = np.concatenate(del_cp_cva,axis=0)
    delta_cp_hedge_full = np.concatenate(del_cp_hedge,axis=0)
    
    weight = delta_cp_full.T.dot(corr).dot(delta_cp_full)

    all_delta_cr[CP_group]={'WS_k(CVA)^2' : (delta_cp_cva_full*delta_cp_cva_full).sum(),
                            'WS_k(Hdg)^2' : (delta_cp_hedge_full*delta_cp_hedge_full).sum(),
                            'WS_k' : delta_cp_full.sum(),
                            'WS_k^2' : weight[0][0]}
                                     
        
# make sure the index matches the correlation matrix                                    
def CR_Bucket(row):
    bucket = row.name
    return int(CR_cpy_buckets.loc['IG_'+bucket]['Bucket'].translate(no_alpha))

bucket_delta = pd.DataFrame(all_delta_cr).T
bucket_delta_cr = bucket_delta.set_index(bucket_delta.apply(CR_Bucket, axis=1))

# bucket_delta_cr = pd.DataFrame(all_delta_cr).T.reindex(CR_correlation.index)

In [ ]:
bucket_delta_cr['Hedge disallowance(R)']=0.01
bucket_delta_cr['K_b']=np.sqrt(
    bucket_delta_cr['WS_k^2']*( 
        1-bucket_delta_cr['Hedge disallowance(R)'])+bucket_delta_cr['Hedge disallowance(R)']*(
        bucket_delta_cr['WS_k(CVA)^2']+bucket_delta_cr['WS_k(Hdg)^2'])
)

In [ ]:
bucket_delta_cr

## Interest Rate Delta

- need to group Interest Rates by Currency
- need to then apply correct Risk weight for ZAR (Domestic), USD, JPY, EUR, GBP, AUD and CAD curves
- once we have the risk weights, need to allow for correlation between different tenors
- Then apply the Hedge disallowance parameter (0.01)

In [ ]:
delta_ir = pd.DataFrame({'Delta_IR':all_buckets.loc[[x for x in all_buckets.index if x.startswith('Delta.IR')]].sum(axis=1)})

#IR_Delta_RW
def RW(row):
    curr, bucket = row.name.split('.')[-2:]
    if curr in IR_Delta_Curr:
        return IR_Delta_RW.loc[bucket]['Risk Weight']
    else:
        return 0.0158
    
delta_ir['RiskWeight'] = delta_ir.apply(RW, axis=1)
delta_ir['SA-WS(CVA)'] = delta_ir['RiskWeight'] * delta_ir['Delta_IR']
delta_ir['SA-WS(Hedge)'] = delta_ir['RiskWeight'] * 0.0
delta_ir['SA-WS'] = delta_ir['SA-WS(CVA)']+delta_ir['SA-WS(Hedge)']

delta_ir_index=delta_ir.set_index(pd.MultiIndex.from_arrays([delta_ir.index.map(lambda x:x.split('.')[2]),delta_ir.index.map(lambda x:x.split('.')[3])]))

In [ ]:
delta_ir
#IR_Delta_RW

In [ ]:
# apply the correlations in IR_Delta_Correlation
delta_ir_ws = {}
n=IR_Delta_Correlation.index.size
for k,v in delta_ir_index.groupby(level=0):
    delta_ir_ws[k]={'WS_k(CVA)^2':(v['SA-WS(CVA)']**2).sum(),
                    'WS_k(Hdg)^2':(v['SA-WS(Hedge)']**2).sum(),
                    'WS_k':v['SA-WS'].sum()}
    
    if k in IR_Delta_Curr:
        block = v.reindex(pd.MultiIndex.from_arrays([[k]*n,IR_Delta_Correlation.index])).fillna(0.0)
        WS = block['SA-WS'].values.reshape(-1,1).dot(block['SA-WS'].values.reshape(1,-1))*IR_Delta_Correlation.values        
        delta_ir_ws[k]['WS_k^2'] = WS.sum()
    else:
        delta_ir_ws[k]['WS_k^2'] = (v['SA-WS']**2).sum()
        
bucket_delta_IR = pd.DataFrame(delta_ir_ws).T

In [ ]:
bucket_delta_IR['K_b']=np.sqrt(bucket_delta_IR['WS_k^2']+R*bucket_delta_IR['WS_k(Hdg)^2'])

bucket_delta_IR

## RiskType cross bucket correlation

Amounts should be aggregated across buckets within each risk class. The correlation
parameters $\gamma_{bc}$ applicable to each risk class are set out below

 - FX is .6 
 - IR is .5.  
 - Credit correlation is specifed in the regs
 - EQ is .15 for buckets 1-10, 
     - .75 for buckets 12 and 13
     - .45 for buckets 12-13 and buckets 1-10
     - 0 for bucket 11 and anything else
 - CM is .2 for buckets 1-10, 0 otherwise
 


$$ Capital_{risktype}=m_{cva}*\sqrt{\sum_b K_b^2+\sum_b \sum_{c\neq b}\gamma_{bc}S_b S_c}$$

Where
$$S_b=max\big(min\big(\sum_{k\in b} WS_k, K_b\big),-K_b\big)$$
$$m_{cva}=1.25$$

In [ ]:
import itertools

def eq_correlation(b,c):
    EQ_Reverse_Buckets
    b1=min(EQ_Reverse_Buckets[b], EQ_Reverse_Buckets[c])
    b2=max(EQ_Reverse_Buckets[b], EQ_Reverse_Buckets[c])
    if b1<11 and b2<11:
        return .15
    if b1==12 and b2==13:
        return .75
    elif b1<=10 and b2>11:
        return .45
    else:
        return 0.0

def apply_correlation(var, corr_fn):
    cross = 0.0
    for i, j in itertools.combinations(var.keys(), 2):
        # default correlations
        y_bc = corr_fn(i,j)
        cross += 2.0 * y_bc * var[i] * var[j]
    return cross

def calc_margin(df, corr, bucket='WS_k'):
    S_b = ((-df['K_b']).clip(lower=df[['K_b', bucket]].min(axis=1))).to_dict()
    w = np.sqrt((df['K_b'] * df['K_b']).sum() + apply_correlation(S_b, corr))
    return w

In [ ]:
MCVA = 1.25

Delta_Risk = pd.DataFrame( {
    'FX':MCVA*calc_margin(bucket_delta_fx, lambda b, c:.6, bucket='SA-WS'),
    'IR':MCVA*calc_margin(bucket_delta_IR, lambda b, c:.5),
    'EQ':MCVA*calc_margin(bucket_delta_eq, eq_correlation, bucket='SA-WS'),
    'CM':MCVA*calc_margin(bucket_delta_cm,  lambda b, c:.2, bucket='SA-WS'),
    'CR':MCVA*calc_margin(bucket_delta_cr, lambda b, c:CR_cpy_correlation.loc[b][c])
    }, index=['Delta'] )

Vega_Risk =  pd.DataFrame( {
    'FX':MCVA*calc_margin(bucket_vega_fx, lambda b, c:.6, bucket='SA-WS'),
    'IR':MCVA*calc_margin(bucket_vega_ir, lambda b, c:.5, bucket='SA-WS'),
    'EQ':MCVA*calc_margin(bucket_vega_eq, eq_correlation, bucket='SA-WS'),
    'CM':MCVA*calc_margin(bucket_vega_cm,  lambda b, c:.2, bucket='SA-WS')
    }, index=['Vega'] )

In [ ]:
Final = pd.concat([Delta_Risk, Vega_Risk], sort=False).T.fillna(0.0)


In [ ]:
Final

In [ ]:
Final.sum(axis=1)

In [ ]:
Final.sum(axis=1).sum()

In [ ]:
Final.sum(axis=1).sum()*1.25

In [ ]:
Final.to_csv('/mnt/MarketData/sensitivities.csv')